# Курсовая работа
## Задание K-3-07. Расчёт движения автомобиля

### Исходные данные
Автомобиль массы $M$, поддерживаемый пружиной с демпфером, перемещается с постоянной
горизонтальной скоростью. В момент времени $t = 0$ вертикальная скорость отсутствует. В
дальнейшем вертикальное смещение дороги от основного уровня описывается функцией
$x_0(t) = 2 (1 - cos7t)$. Предположим, что пружина - линейная с коэффициентом упругости $K$, а
коэффициент демпфирования $r$ является нелинейной функцией относительно двух концов
демпфера:

\begin{equation}
    r = r_0 \Big(1+ c \Big| \frac{dx}{dt} - \frac{dx_0}{dt} \Big| \Big)
\end{equation}

Легко показать, что смещение $x(t)$ центра тяжести автомобиля описывается уравнением:

\begin{equation}
   M\frac{d^2x}{dt^2} = -k (x - x_0(t)) - r \Big( \frac{dx}{dt} - \frac{dx_0(t)}{dt} \Big), \text{    } 0 \le t \le t_{max}
\end{equation}

С начальными условиями $x(0) = 0$, $x'_t(0) = 0$, $t_{max} = 4$ секунды

Значения $c^1$, $c^2$ и $c^3$ являются решением системы уравнений:

\begin{equation}
    \begin{pmatrix}
        46 & 42 & 24 \\
        42 & 49 & 18 \\
        24 & 18 & 16 \\
    \end{pmatrix}
    \cdot
    \begin{pmatrix}
        c^1 \\
        c^2 \\
        c^3 \\
    \end{pmatrix}
    =
    \begin{pmatrix}
        282 \\
        229 \\
        178 \\
    \end{pmatrix}
\end{equation}

$K$ равно:

\begin{equation}
    K = 1235.802 \cdot \int_0^{0.5} \sqrt{\frac{1 - 0.25 x^2}{1 - x^2}} dx
\end{equation}

$M = 25.0449 \cdot x^*$, где $x^*$ - наименьший ненулевой корень уравнения:

\begin{equation}
    2^x - 2x - 1 = 0
\end{equation}

$r_0^1 = 80$, $r_0^2 = 160$ и $r_0^3 = 240$
### Задания
1) Исследовать качество трёх демпферов: $(r_0^1, c^1)$, $(r_0^2, c^2)$, $(r_0^3, c^3)$.

2) Оценить общую погрешность результатов.

3) Варьируя значение $K$ на 5% и значение $M$ на 10%, оценить влияние на точность погрешности исходных данных.

### Выполнение работы

Для начала вычислим значения $c^1$, $c^2$, $c^3$, $M$ и $K$:

In [22]:
from numpy.linalg import solve

A = [[46, 42, 24],
     [42, 49, 18],
     [24, 18, 16]]
b = [282, 229, 178]
x = solve(A, b)
i = 0
for c in x:
    print('c_{i} = {c}'.format(i=i, c=c))
    i += 1

c_0 = -2.0273637840981118e-16
c_1 = 1.0000000000000002
c_2 = 10.0


In [31]:
from scipy.optimize import brentq

M = brentq(lambda t: 2 ** t - 2 * t - 1, 1, 3)
M *= 25.0449
print('M = {M}'.format(M=M))

M = 66.61595721485934


In [29]:
from scipy.integrate import quad

from math import sqrt

K, err = quad(lambda t: sqrt((1 - 0.25 * t ** 2) / (1 - t ** 2)), 0, 0.5)
K *= 1235.802
print('K = {K}'.format(K=K))
print('error = {err}'.format(err=err))

K = 639.9995308637812
error = 5.7496444811892876e-15


Теперь перейдём к решению дифференциального уравнения. Оно второго порядка.

Для начала приведем его к системе из двух уравнений первого порядка при помощи подстановок $y_1(t) = x(t)$ и $y_2(t) = \frac{dx}{dt}$:

\begin{equation}
    \begin{cases}
        y_1'(t) = y_2(t)
        \\
        y_2'(t) = -\frac{K}{M} \Big(y_1(t) - 2(1 - cos7t) \Big) - \frac{r_0}{M} \Big(1 + c \cdot \Big| y_2(t) - 14sin7t \Big| \Big) \Big( y_2(t) - 14sin7t \Big)
    \end{cases}
\end{equation}

In [33]:
from math import sin, cos

def fun(y, t, r, c, k, m):
    return [y[1],
            -k/m * (y[0] - 2 * (1 - cos(7 * t))) - r/m * (1 + c * abs(y[1] - 14 * sin(7 * t)) * (y[1] - 14 * sin(7 * t)))]

In [39]:
from numpy import arange, transpose

from scipy.integrate import odeint

import matplotlib.pyplot as plt

class Damper:
    r : float   
    c : float
    
    def __init__(self, c : float, r : float):
        self.r = r
        self.c = c
    
    def __str__(self):
        return '({r}, {c})'.format(r=self.r, c=self.c)

rs = [80, 160, 240]
dampers = []
for i in range(len(rs)):
    dampers.append(Damper(x[i], rs[i]))

h = 0.001
left = 0
right = 4 + h

points = arange(left, right, h)

initialConditions = [0, 0]

values = []
for d in dampers:
    res, info = odeint(fun, initialConditions, points, args=(d.r, d.c, K, M), atol=1e-8, full_output=True)
    y = transpose(res)[0]
    values.append(y)
    
axes = plt.figure(figsize=(10,7)).gca()

for i in range(len(dampers)):
    axes.plot(points, values[i], linewidth=1.5, label='$D_{i} = {d}$'.format(d=dampers[i], i=i+1))

axes.legend(loc='upper left', fontsize=15)
axes.set_xlabel('$t$', fontsize=20, labelpad=20)
axes.set_ylabel('$y(t)$', fontsize=20, labelpad=20)
axes.tick_params(labelsize=15, pad=10)
axes.set_xlim([left, right])
plt.title('$x(t)$ для различных значений $D = (r, c)$', fontsize=20)
plt.tight_layout()
plt.savefig('./p.png')
plt.show()

In [78]:
from numpy import amax

for i in range(len(points)):
    zeros.append(0)
    

def ro_quad(t, f1, f2):
    sum = 0
    for i in range(len(t)):
        sum += (f1[i] - f2[i]) ** 2
    return sum

def delta(t, f1, f2):
    diff = []
    for i in range(len(t)):
        diff.append(abs(f1[i] - f2[i]))
    return amax(diff)

for i in range(len(values)):
    s = ro_quad(points, values[i], zeros)
    dl = delta(points, values[i], zeros)
    print('D{i}: ro^2 = {s}, delta = {dl}'.format(i=i, s=s, dl=dl))

D0: ro^2 = 26020.911105189694, delta = 4.647877130080668
D1: ro^2 = 18708.94972588545, delta = 3.710034069582049
D2: ro^2 = 21777.93043166616, delta = 3.949934081661019
